In [5]:
import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler

# Variables and loading data

In [6]:
sampling_rate = 256

In [7]:
# Load annotation file
annt = scipy.io.loadmat('annotations_2017.mat')

In [8]:
# Load the data 
signals, signal_headers, header = highlevel.read_edf('eeg5.edf')

# Functions to preprocess

In [9]:
# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

In [10]:
# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

In [11]:
# Combination of all filters and MinMaxScaler
def filter_signals(signals, sampling_rate, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(signals, sampling_rate, notch_frequency, quality_factor)
    scaler = MinMaxScaler()
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

# Functions to label

In [12]:
# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

In [13]:
# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

In [14]:
# Add a time column with the seconds
def add_time(df):
    list_time=[]
    for i in range(len(df)):
        list_time.append(i//sampling_rate)
    df["time"]=list_time
    return df

In [15]:
# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

In [16]:
# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

In [17]:
# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C'], inplace=True)
    return df

In [18]:
# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    add_time(data_patient)
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

# Final preprocessing & label

In [19]:
signals_preprocessed = filter_signals(signals, sampling_rate, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)

In [20]:
d_5 = label_data('eeg5.edf', signals_preprocessed, 5)

In [21]:
d_5

,EEG Fp1-Ref,EEG Fp2-Ref,EEG F7-Ref,EEG F3-Ref,EEG Fz-Ref,EEG F4-Ref,EEG F8-Ref,EEG T3-Ref,EEG C3-Ref,EEG Cz-Ref,...,EEG P3-Ref,EEG Pz-Ref,EEG P4-Ref,EEG T6-Ref,EEG O1-Ref,EEG O2-Ref,ECG EKG,Resp Effort,time,is_seizure_target
0,0.353494,0.793573,1.000000,0.701917,0.393511,0.183450,0.846558,0.579165,0.018523,0.627815,...,0.190606,0.209212,0.769056,0.731333,0.000000,0.179405,0.454318,0.387518,0,1
1,0.336246,0.862974,1.000000,0.638187,0.376305,0.179248,0.838919,0.609300,0.000000,0.605302,...,0.199013,0.214853,0.768280,0.729059,0.017021,0.195751,0.508438,0.393925,0,1
2,0.268712,0.919813,1.000000,0.511203,0.356371,0.224427,0.850186,0.618169,0.000000,0.601201,...,0.191920,0.219181,0.799121,0.757659,0.045152,0.243526,0.520281,0.427653,0,1
3,0.347118,0.805896,1.000000,0.618322,0.364362,0.234248,0.866393,0.616341,0.000000,0.607192,...,0.211140,0.236517,0.800898,0.770108,0.051896,0.248591,0.515687,0.437771,0,1
4,0.425620,0.772917,1.000000,0.615228,0.351910,0.214835,0.839804,0.504690,0.000000,0.602062,...,0.206211,0.235480,0.804234,0.765600,0.037186,0.232421,0.536653,0.433392,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983291,0.358651,0.000000,0.446580,0.350410,0.431132,0.438771,0.431005,0.601553,0.432301,0.466851,...,0.429410,0.481188,0.497280,0.360721,0.479454,0.489994,0.985803,0.412175,3840,0
983292,0.027400,0.011211,0.029130,0.000000,0.017762,0.040012,0.040446,0.011096,0.022597,0.031731,...,0.039975,0.053607,0.071216,0.108301,0.072665,0.062631,1.000000,0.009003,3840,0
983293,0.040683,0.080363,0.029821,0.027907,0.026126,0.036374,0.037631,0.000000,0.028419,0.028432,...,0.037605,0.037596,0.044365,0.081376,0.047479,0.040991,1.000000,0.024221,3840,0
983294,0.966625,1.000000,0.959231,0.976907,0.964426,0.956003,0.956450,0.953622,0.962633,0.956752,...,0.956739,0.947944,0.940503,0.937985,0.941369,0.944088,0.000000,0.968972,3840,0


# Appendix - to remove time and ECG+Resp Effort

In [22]:
def very_clean(df):
    df.drop(columns=['ECG EKG', 'Resp Effort', 'time'], inplace=True)
    return df

In [23]:
d_5 = very_clean(d_5)

In [24]:
d_5.to_csv('patient_5.csv')